In [35]:
data = """jqt: rhn xhk nvd
rsh: frs pzl lsr
xhk: hfx
cmg: qnr nvd lhk bvb
rhn: xhk bvb hfx
bvb: xhk hfx
pzl: lsr hfx nvd
qnr: nvd
ntq: jqt hfx bvb xhk
nvd: lhk
lsr: lhk
rzs: qnr cmg lsr rsh
frs: qnr lhk lsr
"""
data = open('puzzle.data').read()

import re
from collections import defaultdict
from random import choice

def parse(data: str) -> dict[str, list[str]]:
    graph = defaultdict(list)
    for line in data.splitlines():
        node, *nodes = re.findall('[\\w]+', line)
        graph[node] += nodes
        for n in nodes:
            graph[n].append(node)

    return graph

def dump(graph):
    from graphviz import Graph
    from IPython.display import display

    dot = Graph('G')
    edges = set()
    for node, next_nodes in graph.items():
        for next_node in next_nodes:
            if (next_node, node) in edges:
                continue
            edges.add( (node, next_node) )
            dot.edge(node, next_node)

    display(dot)

def karger(graph: dict[str, list[str]]) -> list[tuple[str, str]]:
    while True:
        kgraph = {node: [(n, (node, n)) for n in graph[node]] for node in graph}
        while len(kgraph) > 2:
            n1 = choice(tuple(kgraph))
            n2 = choice(kgraph[n1])[0]
            combined = [(n, v) for n, v in kgraph.pop(n1) + kgraph.pop(n2) if n not in (n1, n2)]
            for node in [n[0] for n in combined]:
                kgraph[node] = [(n1 + n2 if n in (n1, n2) else n, v) for n, v in kgraph[node]]
            kgraph[n1 + n2] = combined
        if len(tuple(kgraph.values())[1]) == 3:
            return [e[1] for e  in tuple(kgraph.values())[0]]

def count_nodes(graph, node):
    open_list = [node]
    visited = set()
    while open_list:
        node = open_list.pop()
        if node in visited:
            continue
        visited.add(node)
        open_list.extend(graph[node])
    return len(visited)

def solve(data):
    graph = parse(data)
    min_edges = karger(graph)
    
    for n1, n2 in min_edges:
        graph[n1].remove(n2)
        graph[n2].remove(n1)
    
    return count_nodes(graph, min_edges[0][0]) * count_nodes(graph, min_edges[0][1])

solve(data)

543036